In [1]:
from pathlib import Path


zarr_path = Path("../sample-data/sample_zarr_nested_groups_from_datatree.zarr")
zarr_path

PosixPath('../sample-data/sample_zarr_nested_groups_from_datatree.zarr')

In [2]:
import xarray as xr

xdt = xr.open_datatree(zarr_path)
xdt


/home/eschalk/work/scientific-data-viewer/.venv/lib/python3.13/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'sentinel-1' loading failed:
No module named 'pkg_resources'
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


<xarray.DataTree>
Group: /
└── Group: /topgroup
    │   Dimensions:   (time: 30)
    │   Coordinates:
    │     * time      (time) datetime64[ns] 240B 2020-01-01 2020-01-02 ... 2020-01-30
    │   Data variables:
    │       metadata  (time) int64 240B ...
    │   Attributes:
    │       title:        Sample xr.DataTree Ocean Data
    │       description:  Sample xr.DataTree Zarr file for testing VSCode extension
    │       institution:  Ocean Test Institute
    │       source:       Generated for testing
    │       history:      Created on 2025-09-29 08:11:35
    │       Conventions:  CF-1.6
    └── Group: /topgroup/ocean_data
        │   Dimensions:      (time: 30, depth: 6, lat: 60, lon: 120)
        │   Coordinates:
        │     * depth        (depth) int64 48B 0 10 20 50 100 200
        │     * lat          (lat) float64 480B -60.0 -57.97 -55.93 ... 55.93 57.97 60.0
        │     * lon          (lon) float64 960B -180.0 -177.0 -173.9 ... 173.9 177.0 180.0
        │   Data variables:
        │       salinity     (time, depth, lat, lon) float64 10MB ...
        │       temperature  (time, depth, lat, lon) float64 10MB ...
        │   Attributes:
        │       description:      Oceanographic measurements
        │       data_type:        oceanographic
        │       collection_date:  2020-01-01
        ├── Group: /topgroup/ocean_data/chemical_properties
        │       Dimensions:    (time: 30, depth: 6, lat: 60, lon: 120)
        │       Data variables:
        │           nitrate    (time, depth, lat, lon) float32 5MB ...
        │           phosphate  (time, depth, lat, lon) float32 5MB ...
        │       Attributes:
        │           description:       Chemical oceanographic properties
        │           measurement_type:  laboratory
        ├── Group: /topgroup/ocean_data/physical_properties
        │       Dimensions:              (time: 30, depth: 6, lat: 60, lon: 120)
        │       Data variables:
        │           temperature_anomaly  (time, depth, lat, lon) float64 10MB ...
        │       Attributes:
        │           description:       Physical oceanographic properties
        │           measurement_type:  in_situ
        └── Group: /topgroup/ocean_data/quality_control
                Dimensions:         (time: 30, depth: 6, lat: 60, lon: 120)
                Data variables:
                    salinity_qc     (time, depth, lat, lon) int8 1MB ...
                    temperature_qc  (time, depth, lat, lon) int8 1MB ...
                Attributes:
                    description:  Quality control flags and information
                    qc_version:   1.2

In [7]:
# type: ignore
import json
from pathlib import PurePosixPath
from typing import Any

from IPython.display import HTML, Markdown
from xarray.core.datatree import DataTree


def show_collapsible_json(dict_to_jsonify: dict[Any, Any], title: str) -> Markdown:
    """
    Take a serializable dict and produce a collapsible markdown section.

    To be used in a notebook.

    Parameters
    ----------
    dict_to_jsonify
        Dict to convert to JSON
    title
        Title that will be shown in the "Summary" section (usage of summary HTML tag)

    Returns
    -------
        collapsible markdown section
    """
    return Markdown(
        f"<details><summary>{title}</summary>\n\n```json\n{json.dumps(dict_to_jsonify,indent=4)}\n\n```\n</details>",
    )


def repr_datatree(
    xdt: DataTree,
    *,
    tabsize: int = 4,
    offset: int = 1,
    with_full_path: bool = False,
    with_group_variables: bool = False,
    rich: bool = False,
    rich_plot: bool = False,
    rich_txt_dataset: bool = True,
    opened: bool = False,
) -> str:
    """
    Represent a DataTree

    Parameters
    ----------
    xdt
        DataTree to represent
    tabsize, optional
        If ``rich`` is OFF, amount of tabs in the text representation, by default 4
    offset, optional
        If ``rich`` is OFF, initial amount of tabs, by default 1
    with_full_path, optional
        Whether to repr the full path or the stem of a Node., by default False
    with_group_variables, optional
        Whether to repr groups only, or groups with their variables, by default False
    rich, optional
        Use rich HTML representation, by default False
    rich_plot, optional
        If ``rich`` is ON, call the default xarray's ``plot`` on the node being currently represented, by default False
    rich_txt_dataset, optional
        If ``rich`` is ON, whether to display text-version or HTML-version of the node being repred, by default True
    opened, optional
        If ``rich`` is ON, whether to initially display opened or closed detail sections, by default False

    Returns
    -------
        String representation
    """
    lines = []

    for node in sorted(xdt.subtree, key=lambda n: n.path):
        path = PurePosixPath(node.path)
        tabs = len(path.parts)
        path_str = f"{(node.name or '') if not with_full_path else path}"
        group_title = f"{path_str} <small> (<code>{path}</code>) </small>"
        if len(node.data_vars) == 0:
            if rich:
                lines.append(Markdown(f"{'#' * tabs} {group_title}"))
            else:
                lines.append(f'{" " * ((tabs - offset) * tabsize)}{group_title}')

            if node.attrs:
                # lines.append(JSON(node.attrs))
                lines.append(
                    Markdown(
                        wrap_in_details(
                            f"```json\n{json.dumps(node.attrs, indent=4)}\n```",
                            summary=f"Attributes <small> (`{path}`) </small>",
                            opened=opened,
                        ),
                    ),
                )

        if with_group_variables:
            for varname in node.ds.data_vars:
                varname_str = f"{path / varname if with_full_path else varname}"
                if rich:
                    if rich_plot:
                        lines.append(Markdown(varname_str))
                        lines.append(node.ds.data_vars[varname].plot())
                else:
                    lines.append(f'{" " * (tabs * tabsize)}{varname_str}')
        elif rich:
            if len(node.data_vars) > 0:
                if rich_txt_dataset:
                    lines.append(
                        HTML(wrap_in_details(node.to_dataset()._repr_html_(), summary=group_title, opened=opened)),
                    )
                    # lines.append(node.ds)
                else:
                    code = f"```python\n{str(node.ds)}\n```"
                    lines.append(Markdown(wrap_in_details(code, summary=group_title, opened=opened)))
    if rich:
        return lines
    else:
        return "\n".join(lines)


def repr_datatree_text(
    xdt: DataTree,
    *,
    tabsize: int = 4,
    offset: int = 1,
    with_full_path: bool = False,
    with_group_variables: bool = False,
) -> str:
    """
    Represent a DataTree as text

    Parameters
    ----------
    xdt
        DataTree to represent
    tabsize, optional
        If ``rich`` is OFF, amount of tabs in the text representation, by default 4
    offset, optional
        If ``rich`` is OFF, initial amount of tabs, by default 1
    with_full_path, optional
        Whether to repr the full path or the stem of a Node., by default False
    with_group_variables, optional
        Whether to repr groups only, or groups with their variables, by default False

    Returns
    -------
        DataTree text representation
    """
    lines = []
    for node in xdt.subtree:
        path = PurePosixPath(node.path)
        tabs = len(path.parts)
        lines.append(f'[ ] {" " * ((tabs - offset) * tabsize)}{str(path).removeprefix("/") if with_full_path else path.stem}')
        if with_group_variables:
            for varname in node.ds.data_vars:
                lines.append(f'[v] {" " * (tabs * tabsize)}{str(path / varname).removeprefix("/") if with_full_path else varname}')
    return "\n".join(lines)


def wrap_in_details(contents: str, *, summary: str | None = None, opened: bool = True) -> str:
    """
    Wrap a string in details

    Parameters
    ----------
    contents
        Input string to wrap
    summary, optional
        Summary in the tag of same name in the detail section, by default None
    opened, optional
        If ``rich`` is ON, whether to initially display opened or closed detail sections, by default False

    Returns
    -------
        HTML-string wrapped with a detail section
    """
    summary_tag = "" if summary is None else f"\n<summary> {summary} </summary>"
    details_prefix = f"<details {'open' if opened else ''}>{summary_tag}\n\n"
    details_suffix = "\n\n</summary>"

    return f"{details_prefix}{contents}{details_suffix}"


In [4]:
display(*repr_datatree(xdt, rich=True, opened=False, rich_txt_dataset=False))
display(*repr_datatree(xdt, rich=True, opened=False, rich_txt_dataset=True))


#  <small> (<code>/</code>) </small>

<details >
<summary> topgroup <small> (<code>/topgroup</code>) </small> </summary>

```python
<xarray.DatasetView> Size: 480B
Dimensions:   (time: 30)
Coordinates:
  * time      (time) datetime64[ns] 240B 2020-01-01 2020-01-02 ... 2020-01-30
Data variables:
    metadata  (time) int64 240B ...
Attributes:
    title:        Sample xr.DataTree Ocean Data
    description:  Sample xr.DataTree Zarr file for testing VSCode extension
    institution:  Ocean Test Institute
    source:       Generated for testing
    history:      Created on 2025-09-29 08:11:35
    Conventions:  CF-1.6
```

</summary>

<details >
<summary> ocean_data <small> (<code>/topgroup/ocean_data</code>) </small> </summary>

```python
<xarray.DatasetView> Size: 21MB
Dimensions:      (time: 30, depth: 6, lat: 60, lon: 120)
Coordinates:
  * time         (time) datetime64[ns] 240B 2020-01-01 2020-01-02 ... 2020-01-30
  * depth        (depth) int64 48B 0 10 20 50 100 200
  * lat          (lat) float64 480B -60.0 -57.97 -55.93 ... 55.93 57.97 60.0
  * lon          (lon) float64 960B -180.0 -177.0 -173.9 ... 173.9 177.0 180.0
Data variables:
    salinity     (time, depth, lat, lon) float64 10MB ...
    temperature  (time, depth, lat, lon) float64 10MB ...
Attributes:
    description:      Oceanographic measurements
    data_type:        oceanographic
    collection_date:  2020-01-01
```

</summary>

<details >
<summary> chemical_properties <small> (<code>/topgroup/ocean_data/chemical_properties</code>) </small> </summary>

```python
<xarray.DatasetView> Size: 10MB
Dimensions:    (time: 30, depth: 6, lat: 60, lon: 120)
Coordinates:
  * time       (time) datetime64[ns] 240B 2020-01-01 2020-01-02 ... 2020-01-30
  * depth      (depth) int64 48B 0 10 20 50 100 200
  * lat        (lat) float64 480B -60.0 -57.97 -55.93 -53.9 ... 55.93 57.97 60.0
  * lon        (lon) float64 960B -180.0 -177.0 -173.9 ... 173.9 177.0 180.0
Data variables:
    nitrate    (time, depth, lat, lon) float32 5MB ...
    phosphate  (time, depth, lat, lon) float32 5MB ...
Attributes:
    description:       Chemical oceanographic properties
    measurement_type:  laboratory
```

</summary>

<details >
<summary> physical_properties <small> (<code>/topgroup/ocean_data/physical_properties</code>) </small> </summary>

```python
<xarray.DatasetView> Size: 10MB
Dimensions:              (time: 30, depth: 6, lat: 60, lon: 120)
Coordinates:
  * time                 (time) datetime64[ns] 240B 2020-01-01 ... 2020-01-30
  * depth                (depth) int64 48B 0 10 20 50 100 200
  * lat                  (lat) float64 480B -60.0 -57.97 -55.93 ... 57.97 60.0
  * lon                  (lon) float64 960B -180.0 -177.0 -173.9 ... 177.0 180.0
Data variables:
    temperature_anomaly  (time, depth, lat, lon) float64 10MB ...
Attributes:
    description:       Physical oceanographic properties
    measurement_type:  in_situ
```

</summary>

<details >
<summary> quality_control <small> (<code>/topgroup/ocean_data/quality_control</code>) </small> </summary>

```python
<xarray.DatasetView> Size: 3MB
Dimensions:         (time: 30, depth: 6, lat: 60, lon: 120)
Coordinates:
  * time            (time) datetime64[ns] 240B 2020-01-01 ... 2020-01-30
  * depth           (depth) int64 48B 0 10 20 50 100 200
  * lat             (lat) float64 480B -60.0 -57.97 -55.93 ... 55.93 57.97 60.0
  * lon             (lon) float64 960B -180.0 -177.0 -173.9 ... 177.0 180.0
Data variables:
    salinity_qc     (time, depth, lat, lon) int8 1MB ...
    temperature_qc  (time, depth, lat, lon) int8 1MB ...
Attributes:
    description:  Quality control flags and information
    qc_version:   1.2
```

</summary>

#  <small> (<code>/</code>) </small>

In [8]:
def boolean_emoji(b: bool) -> str:  
    return "✅" if b else "❌"

for with_group_variables, with_full_path in [(False, False), (True, False), (False, True), (True, True)]:
    print(f"with_group_variables={boolean_emoji(with_group_variables)}, with_full_path={boolean_emoji(with_full_path)}")
    print(repr_datatree_text(xdt, with_group_variables=with_group_variables, with_full_path=with_full_path))
    print()



with_group_variables=❌, with_full_path=❌
[ ] 
[ ]     topgroup
[ ]         ocean_data
[ ]             chemical_properties
[ ]             physical_properties
[ ]             quality_control

with_group_variables=✅, with_full_path=❌
[ ] 
[ ]     topgroup
[v]         metadata
[ ]         ocean_data
[v]             salinity
[v]             temperature
[ ]             chemical_properties
[v]                 nitrate
[v]                 phosphate
[ ]             physical_properties
[v]                 temperature_anomaly
[ ]             quality_control
[v]                 salinity_qc
[v]                 temperature_qc

with_group_variables=❌, with_full_path=✅
[ ] 
[ ]     topgroup
[ ]         topgroup/ocean_data
[ ]             topgroup/ocean_data/chemical_properties
[ ]             topgroup/ocean_data/physical_properties
[ ]             topgroup/ocean_data/quality_control

with_group_variables=✅, with_full_path=✅
[ ] 
[ ]     topgroup
[v]         topgroup/metadata
[ ]         topgroup/ocean

In [6]:
xdt.to_dict().keys()

dict_keys(['/', '/topgroup', '/topgroup/ocean_data', '/topgroup/ocean_data/chemical_properties', '/topgroup/ocean_data/physical_properties', '/topgroup/ocean_data/quality_control'])